# CLASE: Expresiones regulares

__Autor Original:__ Arturo Sánchez Palacio

__Modificado y comentado por:__ Lucía Saiz Lapique

Fecha original: 10/XII/18

Fecha modificación: diciembre de 2019

El objetivo de esta práctica es emplear el RDD access_logs para realizar una serie de cálculos:

En primer lugar se deben cargar los datos en un contexto Spark:

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext()

In [67]:
data_file = "./apache.access.log_small"
raw_data = sc.textFile(data_file)

* El host es el primer elemento
* La fecha es el cuarto elemento (indice 3)
* End point es el indice 6
* Codigo de respuesta el penúltimo elemento (se puede coger poniendo -2)

In [68]:
datos = "./caquita"
raw_datos = sc.textFile(datos)

Hasta aquí se han cargado los datos en el RDD raw_data. Como primera medida para asegurar que los datos se han subido de manera correcta se procede al conteo de los mismos:

In [51]:
raw_data.count()

0

In [69]:
raw_datos.count()

0

Además vamos a visualizar los primero elementos del RDD:

In [5]:
raw_data.take(5)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

Una vez confirmado que la carga ha sido realizada de manera correcta se procede al parseado de los datos:

En el parseado se emplea la expresión regular vista hoy en clase:

In [ ]:
import re
def parse_log1(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([\w\.\s*]+)?\s*"*(\d{3}) (\S+)', line)
    if match is None:
        return 0
    else:
        return 1
n_logs = raw_data.count()

In [30]:
example_rd = raw_data.take(1)

In [31]:
example_rd  ## los null puede que sean guiones. Entre paretesis la peticion que se ha hecho a la web. 

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839']

In [20]:
erd = 'La clase ETL ['

In [21]:
# expresiones regulares:
rexp = '^(\S+)'  # ^ por ahi empieza el string que voy a emplear. le estoy indicando mas o menos como es el elemento por el que quiero que empiece la estructura.
# \S simboliza todo lo que no sea espacio. cuando le pongo el + a cualquier bloque de expresion regular de este tipo indica que eso tiene que aparecer por lo menos una vez. Nada más empezar, tiene que aparecer algo que no sea un espacio.
rexp2 = '^(\S+.\S+.[com])' # ahi le indico que primero tiene que aparecer un elemento q no sea un espacio, luego un punto, luego otra cosa q no sea un espaciom otro punto y una serie de caracteres que formen "com"
rexp3 = '^(\S+) (\S+) (\S+) \[' # los parentesis y los corchetes tienen sentid, asi que si quiero aplicarlo a la expresion regular, no lo aplica, se cree que esta dentro del lenguaje que estoy usando. Si quiero representarlo, tengo que añadir \ antes.

In [23]:
match = re.search(rexp3, erd) # detector de mentiras
match # me dice que es un objeto python del cual puedo extraer por ejemplo: 

<_sre.SRE_Match object; span=(0, 14), match='La clase ETL ['>

In [24]:
match.groups() # si no cumplimos la regla de que obligatoriamente tiene q salir un corchete, por ejemplo, y en vez del corchete hay una letra, nos va a dar error. 

('La', 'clase', 'ETL')

In [25]:
match is None

False

* Intentamos poner lo que nos salio de resultado en example_rd. Como lo que esta entre corchetes no lo queremos parsear ni nada, lo guardamos como si fuese un string completo. 
* Los corchetes sirven para englobar conjuntos de strings o caracteres que pueden aparecer (no obligatorio), pero no más de dos veces. 
* Entre llaves el numero de elementos que se que van a aparecer. Las commillas se pueden poner pero no mezclar simples y dobles; si defines el string gordo con una simple puedes usar la doble sin problema. Si coges la doble para el tocho, habria que poner una barra. 
* Puedo asumir que siempre va a haber un solo espacio pero igual hay mas, (\D = todo lo que no sea dígito), \s*(asterisco) = puede haber un espacio y puede aparecer varias veces (asterisco)*. Si no nos sabemos la estructura, lo ideal es usar un asterisco y asi nos aseguramos de que parsea siempre.
* w = palabra (letras), W (o \w) = todo lo que no fuese una palabra (letras)
* (+) implica que coge varios bloques iguales y obliga a que haya al menos uno.
* Si no pones nada, procesa el primer caracter y nada más. Si estoy seguro de que va a ahaber tres numeros, le puedo poner \d{3} (d porque son digitos) y me va a devolver los tres primeros numeros 
* Cheat sheet

In [47]:
rexp4 = '^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"*(\d{3})(\S+)'

In [48]:
match2 = re.search(rexp4, example_rd[0])

In [49]:
match2.groups() # todo lo que este entre parentesis va a ser un elemento de la lista que devuelve. 

('in24.inetnebr.com',
 '-',
 '-',
 '01/Aug/1995:00:00:01',
 '0400',
 'GET',
 '/shuttle/missions/sts-68/news/sts-68-mcc-05.txt',
 'HTTP/1.0"',
 '200 ',
 '183',
 '9')

In [7]:
def parse_log2(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    if match is None:
        return (line, 0)
    else:
        return (line, 1)


In [8]:
def map_log(line):
    match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*(\S+)\s*(\S+)\s*([/\w\.\s*]+)?\s*"* (\d{3}) (\S+)',line)
    if match is None:
        match = re.search('^(\S+) (\S+) (\S+) \[(\S+) [-](\d{4})\] "(\S+)\s*([/\w\.]+)>*([\w/\s\.]+)\s*(\S+)\s*(\d{3})\s*(\S+)',line)
    return(match.groups())
parsed_rdd = raw_data.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 1).map(lambda line : line[0]) #map con el segundo parseador, filtramos cuales de ellos han sido uno (que el paseador los pasa adecuadamente) y despues mapeamos
parsed_def = parsed_rdd.map(lambda line: map_log(line))

In [16]:
raw_data.map(lambda line: parse_log1(line)).filter(lambda booll: bool == 0).count()

0

In [17]:
raw_data.map(lambda line: parse_log2(line)).filter(lambda line: line[1] == 0).take(1)

[]

A continuación se presentan las estadísticas (incluyendo mínimo, máximo y media) del tamaño de las peticiones:

In [59]:
def convert_long(x):
    x = re.sub('[^0-9]',"",x) 
    if x =="":
        return 0
    else:
        return int(x)
parsed_def.map(lambda line: convert_long(line[-1])).stats() # stats nos da un summary de los datos, tiene que tener formato nuerico

(count: 3432, mean: 16051.863636363621, stdev: 53247.8157482, max: 887988.0, min: 0.0)

El siguiente objetivo es calcular el número de peticiones de cada código de respuesta:

In [60]:
n_codes = parsed_def.map(lambda line: (line[-2], 1)).distinct().count()
codes_count = (parsed_def.map(lambda line: (line[-2], 1))
          .reduceByKey(lambda a, b: a + b) # nos permite contar cuantos unos hay. podemos tb hacer directamente un count
          .takeOrdered(n_codes, lambda x: -x[1]))  # si no ponemos nunguna funcion, por defecto pondra la identidad 
codes_count

[('200', 3140), ('304', 219), ('302', 50), ('404', 22), ('403', 1)]

__Solución:__ Así el código 200 aparece para 3140 logs, el 304 para 219 logs, el 302 para 50 logs, el 404 para 22 logs y el 403 para un único log.

Si se desean ver  20 hosts visitados más de diez veces:

In [62]:
result = parsed_def.map(lambda line: (line[0],1)).reduceByKey(lambda a, b: a + b).takeOrdered(20, lambda x: -x[1])
result

[('ix-min1-02.ix.netcom.com', 78),
 ('uplherc.upl.com', 71),
 ('port26.ts1.msstate.edu', 59),
 ('h96-158.ccnet.com', 56),
 ('in24.inetnebr.com', 55),
 ('piweba3y.prodigy.com', 54),
 ('thing1.cchem.berkeley.edu', 54),
 ('adam.tower.com.au', 44),
 ('ip55.van2.pacifier.com', 43),
 ('ppp1016.po.iijnet.or.jp', 41),
 ('hsccs_gatorbox07.unm.edu', 40),
 ('www-b2.proxy.aol.com', 40),
 ('www-d1.proxy.aol.com', 39),
 ('133.43.96.45', 37),
 ('port13.wavenet.com', 37),
 ('pc-heh.icl.dk', 33),
 ('haraway.ucet.ufl.edu', 32),
 ('193.84.66.147', 31),
 ('www-c1.proxy.aol.com', 30),
 ('143.158.26.50', 29)]

In [64]:
result = parsed_def.map(lambda line: (line[0],1)).reduceByKey(lambda a, b: a + b) # el reduce te lo trae pero conuna funcion que tu le pases
result.filter(lambda x: x[1] > 10).takeOrdered(20, lambda x: -x[1])

[('ix-min1-02.ix.netcom.com', 78),
 ('uplherc.upl.com', 71),
 ('port26.ts1.msstate.edu', 59),
 ('h96-158.ccnet.com', 56),
 ('in24.inetnebr.com', 55),
 ('piweba3y.prodigy.com', 54),
 ('thing1.cchem.berkeley.edu', 54),
 ('adam.tower.com.au', 44),
 ('ip55.van2.pacifier.com', 43),
 ('ppp1016.po.iijnet.or.jp', 41),
 ('hsccs_gatorbox07.unm.edu', 40),
 ('www-b2.proxy.aol.com', 40),
 ('www-d1.proxy.aol.com', 39),
 ('133.43.96.45', 37),
 ('port13.wavenet.com', 37),
 ('pc-heh.icl.dk', 33),
 ('haraway.ucet.ufl.edu', 32),
 ('193.84.66.147', 31),
 ('www-c1.proxy.aol.com', 30),
 ('143.158.26.50', 29)]

A continuación se muestran los 10 endpoints más visitados:

In [12]:
result = parsed_def.map(lambda line: (line[6],1)).reduceByKey(lambda a, b: a + b).takeOrdered(10, lambda x: -x[1])
result # todos son mayores que 10 

[('/images/KSC-logosmall.gif', 167),
 ('/images/NASA-logosmall.gif', 160),
 ('/images/MOSAIC-logosmall.gif', 122),
 ('/images/WORLD-logosmall.gif', 120),
 ('/images/USA-logosmall.gif', 118),
 ('/images/ksclogo-medium.gif', 106),
 ('/', 85),
 ('/history/apollo/images/apollo-logo1.gif', 74),
 ('/images/launch-logo.gif', 69),
 ('/images/ksclogosmall.gif', 66)]

Si se desean los 10 endpoints más visitados que no han devuelto un resultado (es decir, que tienen un código distinto de 200):

In [13]:
result = (parsed_def.filter(lambda line: line[9] != '200')  # tu no tienes la variable por la que quieres filtrar. El filtro negativo se hace con esa sintaxis
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b)
          .takeOrdered(10, lambda x: -x[1]))
result

[('/images/NASA-logosmall.gif', 25),
 ('/images/KSC-logosmall.gif', 24),
 ('/images/MOSAIC-logosmall.gif', 17),
 ('/images/WORLD-logosmall.gif', 17),
 ('/images/USA-logosmall.gif', 16),
 ('/images/ksclogo-medium.gif', 10),
 ('/software/winvn/bluemarb.gif', 8),
 ('/software/winvn/winvn.html', 8),
 ('/images/construct.gif', 8),
 ('/software/winvn/wvsmall.gif', 6)]

A continuación se calcula el número de hosts distintos:

In [14]:
parsed_def.map(lambda line: line[0]).distinct().count()

311

Tras ello se puede calcular el número de hosts únicos cada día:

In [15]:
from datetime import datetime   
def day_month(line):
    date_time = line[3]  # sabemos que la fecha es el cuarto elemento empezando por 0
    return datetime.strptime(date_time[:11], "%d/%b/%Y") #Se parsea la fecha para trabajar con ella tal y como vimos en clase.Cogemos todos los elementos del string hasta el 11 e introducimos el formato (d = dias, b = mes en formato string(las tres primeras letras) y Y = año)
result = parsed_def.map(lambda line:  (day_month(line), 1)).reduceByKey(lambda a, b: a + b).distinct().collect()
result
# el objeto es complejo; 


[(datetime.datetime(1995, 8, 1, 0, 0), 3432)]

__Nota:__ En este dataset solo se poseén datos de un día por esa razón se obtiene este resultado.

Tras esto se pide calcular la media de peticiones diaria por host.

In [70]:
import pandas as pd

In [71]:
unique_result = (parsed_def.map(lambda line:  (day_month(line),line[0])) # ¿cuantas peticiones hay por dia y host?
          .groupByKey().mapValues(set)
          .map(lambda x: (x[0], len(x[1])))) # en vez de una lista es un set de datos en python

length_result = (parsed_def.map(lambda line:  (day_month(line),line[0])) # 
          .groupByKey().mapValues(len)) # otra opcion es hacer el length directamente

joined = length_result.join(unique_result).map(lambda a: (a[0], (a[1][0])/(a[1][1]))).collect()
day = [x[0] for x in joined]
count = [x[1] for x in joined]
day_count_dct = {'Día':day, 'Media':count}
day_count_df = pd.DataFrame(day_count_dct )


El panda en este caso no se usa para realizar ningún cálculo si no simplemente para una mejor visualización.

__Solución:__ 

In [18]:
day_count_df

,Día,Media
0,1995-08-01,11.03537


A continuación, se muestra una lista de 40 endpoints que generan código de respuesta = 404:

In [19]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b).distinct()
          .takeOrdered(40, lambda x: -x[1])) # parentesis para que considere que todo es un bloque y te deje hacer intro
result

[('/history/apollo/a-001/a-001-patch-small.gif', 4),
 ('/pub/winvn/release.txt', 4),
 ('/history/apollo/a-004/a-004-patch-small.gif', 2),
 ('/history/apollo/a-001/movies/', 2),
 ('/pub/winvn/readme.txt', 2),
 ('/www/software/winvn/winvn.html', 1),
 ('/elv/DELTA/uncons.htm', 1),
 ('/shuttle/resources/orbiters/discovery.gif', 1),
 ('/history/apollo/a-001/images/', 1),
 ('/sts-71/launch/', 1),
 ('/history/apollo/apollo-13.html', 1),
 ('/history/history.htm', 1),
 ('/history/apollo/a-004/movies/', 1)]

__Nota:__ En la lista solo aparecen 13 endpoints porque se está trabajando con la base de datos reducida.

El siguiente comando mostraría el top 25 de endpoints que más códigos 404 generan de dispnerse de más de 13:

In [20]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line: (line[6], 1))
          .reduceByKey(lambda a, b: a+b).distinct()
          .takeOrdered(25, lambda x: -x[1]))
result

[('/history/apollo/a-001/a-001-patch-small.gif', 4),
 ('/pub/winvn/release.txt', 4),
 ('/history/apollo/a-004/a-004-patch-small.gif', 2),
 ('/history/apollo/a-001/movies/', 2),
 ('/pub/winvn/readme.txt', 2),
 ('/www/software/winvn/winvn.html', 1),
 ('/elv/DELTA/uncons.htm', 1),
 ('/shuttle/resources/orbiters/discovery.gif', 1),
 ('/history/apollo/a-001/images/', 1),
 ('/sts-71/launch/', 1),
 ('/history/apollo/apollo-13.html', 1),
 ('/history/history.htm', 1),
 ('/history/apollo/a-004/movies/', 1)]

Si se desean obtener el top 5 de días que más códigos de error 404 generan:

In [21]:
result = (parsed_def.filter(lambda line: line[9] == '404')
          .map(lambda line:  (day_month(line), 1))
          .reduceByKey(lambda a, b: a+b).collect())
day = [x[0] for x in result]
count = [x[1] for x in result]
day_count_dct = {'day':day, 'count':count}
day_count_df = pd.DataFrame(day_count_dct )


De nuevo el paso a DF se emplea únicamente para una mejor visualización:

In [22]:
day_count_df.sort_values('count', ascending = False)[:10]


,count,day
0,22,1995-08-01


__Nota:__ De nuevo se obtiene un solo día por trabajarse con la base de datos reducida.

In [73]:
from pyspark.sql import SQLContext, Row # Row nos permite darle nombres a cada elemento de la lista
sqlContext = SQLContext(sc)

In [74]:
sql_data = parsed_def.map(lambda p: 
                         Row(Host = p[0], 
                             date = datetime.strtdate(p[3][:11], "%d%b%Y"),
                            endpoint = p[6], code = p[-2],
                            size = p[-1]))

In [75]:
sql_data.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 44.0 failed 1 times, most recent failure: Lost task 0.0 in stage 44.0 (TID 46, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1306, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-74-b172e1d19d02>", line 3, in <lambda>
AttributeError: type object 'datetime.datetime' has no attribute 'strtdate'

	at o